In [1]:
from veg2hab.criteria import FGRCriterium, LBKCriterium, BodemCriterium, EnCriteria, NietCriterium, OfCriteria, OudeBossenCriterium
from veg2hab.enums import FGRType, LBKType, BodemType, OBKWaarden
import pandas as pd
import random

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


FGR

In [2]:
du = FGRCriterium(wanted_fgrtype=FGRType.DU)

for fgr_type in [FGRType.DU, FGRType.HL, pd.NA]:
    row = pd.Series({"fgr": fgr_type, "fgr_percentage": random.randint(0, 100)})
    du.check(row)
    print(fgr_type)
    print(du.evaluation)
    print(du.get_info())
    print("--------")

FGRType.DU
TRUE
{'FGR type is Duinen (88.0%).'}
--------
FGRType.HL
FALSE
{'FGR type is niet Duinen, maar Heuvelland (29.0%).'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één FGR-vlak.'}
--------


Bodemkaart met enkel negatieven

In [3]:
lv = BodemCriterium(wanted_bodemtype=BodemType.LEEMARME_VAAGGRONDEN_H9190)

for bodem_code in [["Zn21"], ["Abcd"], pd.NA]:
    row = pd.Series({"bodem": bodem_code, "bodem_percentage": random.randint(0, 100)})
    lv.check(row)
    print(bodem_code)
    print(lv.evaluation)
    print(lv.get_info())
    print("--------")

['Zn21']
CANNOT_BE_AUTOMATED
{'Dit is mogelijk Leemarme vaaggronden want bodemcode Zn21 (43.0%).'}
--------
['Abcd']
FALSE
{'Dit is niet Leemarme vaaggronden want bodemcode Abcd (45.0%).'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet binnen een bodemkaartvlak.'}
--------


Bodemkaart met zowel positieven als negatieven

In [4]:
vg = BodemCriterium(wanted_bodemtype=BodemType.VAAGGRONDEN)

for bodem_code in [["Zn21"], ["Abcd"], pd.NA]:
    row = pd.Series({"bodem": bodem_code, "bodem_percentage": random.randint(0, 100)})
    vg.check(row)
    print(bodem_code)
    print(vg.evaluation)
    print(vg.get_info())
    print("--------")

['Zn21']
TRUE
{'Dit is Vaaggronden want bodemcode Zn21 (73.0%).'}
--------
['Abcd']
FALSE
{'Dit is niet Vaaggronden want bodemcode Abcd (29.0%).'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet binnen een bodemkaartvlak.'}
--------


LBK met enkel negatieven

In [5]:
hh = LBKCriterium(wanted_lbktype=LBKType.HERSTELLEND_HOOGVEEN)

for lbk_code in ["HzHL", "Abcd", pd.NA]:
    row = pd.Series({"lbk": lbk_code, "lbk_percentage": random.randint(0, 100)})
    hh.check(row)
    print(lbk_code)
    print(hh.evaluation)
    print(hh.get_info())
    print("--------")

HzHL
CANNOT_BE_AUTOMATED
{'Dit is mogelijk Herstellend hoogveen want LBK code HzHL (62.0%).'}
--------
Abcd
FALSE
{'Dit is niet Herstellend hoogveen want LBK code Abcd (93.0%).'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één LBK-vak'}
--------


LBK met enkel positieven

In [14]:
hl = LBKCriterium(wanted_lbktype=LBKType.HOOGVEENLANDSCHAP)

for lbk_code in ["HzHL", "Abcd", pd.NA]:
    row = pd.Series({"lbk": lbk_code, "lbk_percentage": random.randint(0, 100)})
    hl.check(row)
    print(lbk_code)
    print(hl.evaluation)
    print(hl.get_info())
    print("--------")

HzHL
TRUE
{'Dit is Hoogveenlandschap want LBK code HzHL (44.0%).'}
--------
Abcd
CANNOT_BE_AUTOMATED
{'Dit is mogelijk toch Hoogveenlandschap ondanks LBK code Abcd (60.0%).'}
--------
<NA>
CANNOT_BE_AUTOMATED
{'Dit vlak ligt niet mooi binnen één LBK-vak'}
--------


Oude Bossenkaart

In [7]:
obk = OudeBossenCriterium(for_habtype="H9120")

for obk_waarde in [pd.NA, OBKWaarden(H9120=1, H9190=1), OBKWaarden(H9120=0, H9190=2)]:
    row = pd.Series({"obk": obk_waarde, "obk_percentage": random.randint(0, 100)})
    obk.check(row)
    print(obk_waarde)
    print(obk.evaluation)
    print(obk.get_info())
    print("--------")

<NA>
FALSE
{'Dit is geen oud bos, want niet binnen boskaartvlak.'}
--------
H9120=1 H9190=1
CANNOT_BE_AUTOMATED
{'Dit is mogelijk oud bos, want binnen boskaartvlak (H9120: 1, H9190: 1) (87.0%).'}
--------
H9120=0 H9190=2
FALSE
{'Dit is geen oud bos, want binnen boskaartvlak (H9120: 0, H9190: 2) (29.0%).'}
--------


Combineren

In [8]:
crit = EnCriteria(
    sub_criteria=
    [
        OfCriteria(
            sub_criteria=
            [
                LBKCriterium(wanted_lbktype=LBKType.HERSTELLEND_HOOGVEEN),
                LBKCriterium(wanted_lbktype=LBKType.HOOGVEENLANDSCHAP),
                BodemCriterium(wanted_bodemtype=BodemType.LEEMARME_HUMUSPODZOLGRONDEN),
            ]), 
        NietCriterium(
            sub_criterium=FGRCriterium(wanted_fgrtype=FGRType.LV)
        )
    ]
)

for lbk_code, fgr_code, bodem_code in [("HzHL", FGRType.DU, ["Zn21"]), ("Abcd", FGRType.LV, ["Abcd"]), (pd.NA, pd.NA, pd.NA)]:
    row = pd.Series({"lbk": lbk_code, "lbk_percentage": random.randint(0, 100), "fgr": fgr_code, "fgr_percentage": random.randint(0, 100), "bodem": bodem_code, "bodem_percentage": random.randint(0, 100)})
    crit.check(row)
    print(str(crit))
    print(crit.evaluation)
    for sentence in crit.get_info():
        print(sentence)
    print("--------")

((LBK is Herstellend hoogveen (C) of LBK is Hoogveenlandschap (T) of Bodem is Leemarme humuspodzolgronden (F)) en niet FGR is Laagveengebied (T))
TRUE
Dit is mogelijk Herstellend hoogveen want LBK code HzHL (38.0%).
Dit is Hoogveenlandschap want LBK code HzHL (38.0%).
FGR type is niet Laagveengebied, maar Duinen (25.0%).
Dit is niet Leemarme humuspodzolgronden want bodemcode Zn21 (24.0%).
--------
((LBK is Herstellend hoogveen (F) of LBK is Hoogveenlandschap (C) of Bodem is Leemarme humuspodzolgronden (F)) en niet FGR is Laagveengebied (F))
FALSE
FGR type is Laagveengebied (72.0%).
Dit is niet Herstellend hoogveen want LBK code Abcd (0.0%).
Dit is niet Leemarme humuspodzolgronden want bodemcode Abcd (81.0%).
Dit is mogelijk toch Hoogveenlandschap ondanks LBK code Abcd (0.0%).
--------
((LBK is Herstellend hoogveen (C) of LBK is Hoogveenlandschap (C) of Bodem is Leemarme humuspodzolgronden (C)) en niet FGR is Laagveengebied (C))
CANNOT_BE_AUTOMATED
Dit vlak ligt niet mooi binnen één FGR